# Review Classification with Yelp Open Dataset

This notebook contains a code which you can execute to make a rating prediction on a review of your choice. The model used is a neural network built with Tensorflow and trained on the data of the [Yelp Open Dataset](https://www.yelp.com/dataset); the text of the reviews is processed with some NLP techniques explained [here](https://www.youtube.com/watch?v=fNxaJsNG3-s&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lorenzovngl/analysis-of-yelp-open-dataset/blob/master/notebooks/review_model_test.ipynb)

In [ ]:
# Parameters for word embedding

vocab_size = 10000
embedding_dim = 128
max_length = 256

In [ ]:
# Download train sentences

!wget --no-check-certificate \
    https://raw.githubusercontent.com/lorenzovngl/analysis-of-yelp-open-dataset/master/notebooks/models/train_sentences.json \
    -O /tmp/train_sentences.json

In [ ]:
# Load training sentences to setup the tokenizer

import json
from tensorflow.keras.preprocessing.text import Tokenizer

with open("/tmp/train_sentences.json", 'r') as f:
    train_sentences = json.load(f)
f.close()

tokenizer = Tokenizer(num_words=vocab_size, oov_token="OOV")
tokenizer.fit_on_texts(train_sentences)

In [ ]:
# Libraries needed

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [ ]:
# Download weight for the model

!wget --no-check-certificate \
    https://raw.githubusercontent.com/lorenzovngl/analysis-of-yelp-open-dataset/master/notebooks/models/model_weights_run2.h5 \
    -O /tmp/model_weights_run2.h5

In [ ]:
# Configure the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.load_weights('/tmp/model_weights_run2.h5')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Replace sentences with ones of which you want to know prediction (in English only)

sentences = [
    "Horrible, this is a bad place. Disgusting.",
    "I recommend to all to have a dinner here.",
    "This is the best place I have ever been!",
    "Not so good, not so bad.",
    "Not so bad, not so good."
]

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
# Make the predicion and print results

result = model.predict(padded)

for i in range(len(result)):    
    fig, axes = plt.subplots(1, 1)
    fig.suptitle('"%s" rating prediction' % sentences[i], fontsize=16)

    axes.bar([k+1 for k in range(5)], [result[i][j] for j in range(5)])
    axes.set_ylabel('Probability')
    axes.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    axes.set_xlabel('Stars')
    axes.set_ylim(0, 1)
    for h, v in enumerate(result[i]):
        axes.text(h+1, v+0.05, ("%.2f%%" % float(v*100)), horizontalalignment='center')

    plt.show()